<a href="https://colab.research.google.com/github/itsjean2/data-course-sample/blob/main/AC_Week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [260]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [261]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 15:35:37--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.2’

All_Beauty.csv.2    100%[===================>]  14.78M  8.25MB/s    in 1.8s    

2021-12-26 15:35:39 (8.25 MB/s) - ‘All_Beauty.csv.2’ saved [15499476/15499476]

--2021-12-26 15:35:39--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.2’

meta_All_Beauty.jso 100%[===================>]   9.85M  6.02MB/s    in 1.6s   

In [262]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [263]:
metadata.head()
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32892 entries, 0 to 32891
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   category         32892 non-null  object
 1   tech1            32892 non-null  object
 2   description      32892 non-null  object
 3   fit              32892 non-null  object
 4   title            32892 non-null  object
 5   also_buy         32892 non-null  object
 6   tech2            32892 non-null  object
 7   brand            32892 non-null  object
 8   feature          32892 non-null  object
 9   rank             32892 non-null  object
 10  also_view        32892 non-null  object
 11  details          32892 non-null  object
 12  main_cat         32892 non-null  object
 13  similar_item     32892 non-null  object
 14  date             32892 non-null  object
 15  price            32892 non-null  object
 16  asin             32892 non-null  object
 17  imageURL         32892 non-null

In [264]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [265]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
ratings=ratings.sort_values('overall',ascending=False)
ratings.head()

,asin,reviewerID,overall,unixReviewTime,DATE
157770,B00IIAK0RW,AFJZW12SLX9OE,5.0,1441756800,2015-09-09
267963,B001P5HCKK,AAQT6LPLDRT14,5.0,1464566400,2016-05-30
348150,B019DS9CD0,AHTEJWA46J23Y,5.0,1470873600,2016-08-11
158513,B00IPVCCVQ,A2584GN5TUEUO7,5.0,1421712000,2015-01-20
158514,B00IPVCCVQ,A1AF38ORF02M2P,5.0,1421625600,2015-01-19


In [266]:
import numpy as np
meta_select=np.array([metadata['asin'],metadata['also_buy'],metadata['also_view'],metadata['rank'],metadata['brand'],metadata['similar_item'],metadata['price']]).T
pdmetadata=pd.DataFrame(meta_select,columns=["asin","also_buy","also_view","rank","brand","similar_item","price"])
#pdmetadata=pdmetadata.dropna()
pdmetadata=pdmetadata.astype('str').drop_duplicates()
pdmetadata=pdmetadata.replace('[]', np.nan)
pdmetadata['price'] = pdmetadata['price'].str.replace('$', '')
pdmetadata['rank'] = pdmetadata['rank'].str.replace(',', '')
pdmetadata['rank'] = pd.to_numeric(pdmetadata['rank'].str.extract(r'(\d+)')[0])
pdmetadata=pdmetadata.sort_values('rank',ascending=True)
#pdmetadata=pdmetadata.dropna()
pdmetadata.info()
pdmetadata.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32488 entries, 8540 to 32669
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   asin          32488 non-null  object 
 1   also_buy      6557 non-null   object 
 2   also_view     8002 non-null   object 
 3   rank          32149 non-null  float64
 4   brand         32488 non-null  object 
 5   similar_item  32488 non-null  object 
 6   price         32488 non-null  object 
dtypes: float64(1), object(6)
memory usage: 2.0+ MB


,asin,also_buy,also_view,rank,brand,similar_item,price
8540,B00BXPVIT8,NaN,"['B07L9PW8R3', 'B000KND8IS', 'B0786ZZQK6', 'B0...",35.0,Braun,,21.44
6268,B006IB5T4W,NaN,"['B005UEB96K', 'B07HDDRBX8', 'B00E3H3260', 'B0...",65.0,Aquaphor,,11.29
12028,B00HYEPVG8,NaN,"['B00DQ2W7UK', 'B00PKHIDRA', 'B07MB4LZC4', 'B0...",245.0,Astra,,9.59
3606,B001QY8QXM,NaN,"['B00DQ2W7UK', 'B00PKHIDRA', 'B07CGMTVM4', 'B0...",280.0,Astra,,9.30
77,B00005JS5C,NaN,"['B00TYSQXIC', 'B0018A32XS', 'B000G0GW6K', 'B0...",457.0,Panasonic,,18.49


## 資料切分
我們會利用「時間」作為切分的條件，將資料分成「最後一個月以前的資料」與「最後一個月的真實購買資料」

In [267]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())
print(users)

['A100XQFWKQ30O2', 'A103T1QOGFCSEH', 'A106UKKSJ2KXPF', 'A10A7GV4D5A11V', 'A1119JJ37ZLB8R', 'A113UOOLBSZN52', 'A12M4U7WK4ALCR', 'A12T8YTW6VWT7S', 'A1364JXGKB46MM', 'A137DALOQFKBTI', 'A13FEZ3WV7S2EY', 'A13IV4I1B0RXMG', 'A13JU88JAHN72I', 'A13K55R6VH1OOD', 'A13P7VFU075A', 'A13SWYE4QLB6NG', 'A13ZTQ0Q4ATA41', 'A142EDN04OD62U', 'A142I22FIC8MZK', 'A14834QTII5TLT', 'A14A447VPACTBC', 'A14AP6MN5XO6LB', 'A14CLF25IX25US', 'A14LYXC3HTBAHI', 'A14VUW4KZ34EOE', 'A14Y32P26G9YL', 'A157T25PBS7MX4', 'A15HZDSERD85C8', 'A15JJ8J1FGADIX', 'A15ZCL70JXXH89', 'A1617KN2IAWZ6J', 'A16E0O88262HKA', 'A16NSZ58PTVIYF', 'A16UGDXRTDLJG5', 'A16X9HR3UFQQXY', 'A16Y7V1CZCWKFV', 'A174YOBOSW9WDN', 'A1786SKRAJXH86', 'A17K2BUZ20WD2', 'A17LYRFV645L0V', 'A18LNGVXDZBTUR', 'A19503XX7GU6J2', 'A19HVHRZDYFEOP', 'A19JM38B861BO4', 'A19KGY3W1P0MHT', 'A19KJPOF3QOJ24', 'A19U5RT4Q8Q1T6', 'A1AQNQUNX8B6JT', 'A1ARU0IN6RUFFD', 'A1BB4635KSKDMV', 'A1BCC7X6KRTBG0', 'A1BPJ56BXENC7', 'A1C0RI6W65FC8B', 'A1C60LJGZVEK27', 'A1CB2SMS3TD44Z', 'A1CNSU8P68CPL

In [268]:
#合併評分和訓練購買紀錄
recommand=pd.merge(ratings_trainings,pdmetadata,left_on='asin', right_on='asin').drop_duplicates()
recommand.info()
recommand.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356562 entries, 0 to 365629
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   asin            356562 non-null  object        
 1   reviewerID      356562 non-null  object        
 2   overall         356562 non-null  float64       
 3   unixReviewTime  356562 non-null  int64         
 4   DATE            356562 non-null  datetime64[ns]
 5   also_buy        145936 non-null  object        
 6   also_view       209363 non-null  object        
 7   rank            354243 non-null  float64       
 8   brand           356562 non-null  object        
 9   similar_item    356562 non-null  object        
 10  price           356562 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 32.6+ MB


,asin,reviewerID,overall,unixReviewTime,DATE,also_buy,also_view,rank,brand,similar_item,price
0,B00IIAK0RW,AFJZW12SLX9OE,5.0,1441756800,2015-09-09,"['B014G8IYRM', 'B014LYHMOM', 'B0011TWSJK', 'B0...","['B014G8IYRM', 'B014LYHMOM', 'B071JQRQG8']",677549.0,SpellboundRX,,
1,B00IIAK0RW,A3SGO0CKI745M6,5.0,1454457600,2016-02-03,"['B014G8IYRM', 'B014LYHMOM', 'B0011TWSJK', 'B0...","['B014G8IYRM', 'B014LYHMOM', 'B071JQRQG8']",677549.0,SpellboundRX,,
2,B00IIAK0RW,A3JVNZBGDH545P,5.0,1452988800,2016-01-17,"['B014G8IYRM', 'B014LYHMOM', 'B0011TWSJK', 'B0...","['B014G8IYRM', 'B014LYHMOM', 'B071JQRQG8']",677549.0,SpellboundRX,,
3,B00IIAK0RW,A30E4KKYKZS0BY,5.0,1452297600,2016-01-09,"['B014G8IYRM', 'B014LYHMOM', 'B0011TWSJK', 'B0...","['B014G8IYRM', 'B014LYHMOM', 'B071JQRQG8']",677549.0,SpellboundRX,,
4,B00IIAK0RW,AL8AONBNUM9V9,5.0,1451952000,2016-01-05,"['B014G8IYRM', 'B014LYHMOM', 'B0011TWSJK', 'B0...","['B014G8IYRM', 'B014LYHMOM', 'B071JQRQG8']",677549.0,SpellboundRX,,


In [269]:
#其他人也買過清單
alsobuy=pd.DataFrame((np.array([recommand['asin'],recommand['also_buy']]).T),columns=['asin','also_buy'])
alsobuy.index=recommand['reviewerID']
alsobuy=alsobuy.dropna().drop_duplicates()

for i in alsobuy.index:
 if i=='A1B3CS4QHXD0UU':
  print(int(len(alsobuy['also_buy'][i])/14))
alsobuy.head()

2


,asin,also_buy
reviewerID,,
AFJZW12SLX9OE,B00IIAK0RW,"['B014G8IYRM', 'B014LYHMOM', 'B0011TWSJK', 'B0..."
AHTEJWA46J23Y,B019DS9CD0,"['B008UAVXP8', 'B01H7TVLW2', 'B01MR2JKTH']"
A2584GN5TUEUO7,B00IPVCCVQ,"['B00FBVRR4U', 'B01NADN8II', 'B00FKVFX60', 'B0..."
A15UIQI610SE61,B00W2Y6TI4,"['B00UZ9503S', 'B00XW05X20', 'B07MB9K7XD', 'B0..."
A1B3CS4QHXD0UU,B00IU5EDKK,"['B00ER5NN3K', 'B01CILZXWI']"


## 產生推薦

In [270]:
#評分最高的清單,篩選最高分且為近一年的評分紀錄
rank=recommand.iloc[:,0:5]
rank.index=rank['reviewerID']
rank=rank.sort_values(['overall','DATE'],ascending=False)
mask=(rank['overall']==5) & (rank['DATE']>= '2018-01-01')
rank=rank[mask]
rank['asin'][:5].tolist()


['B01DKQAXC0', 'B01DKQAXC0', 'B01AR19RZU', 'B012Z7IHHI', 'B015ZOWER2']

In [271]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    for user in users:
     if user in alsobuy.index:
      rankadd=k-int((len(alsobuy['also_buy'])/14))
      list(alsobuy['also_buy'][user]).extend(rank['asin'][:rankadd].tolist())
      recommendations[user] =alsobuy['also_buy'][user]
     else:
      recommendations[user] = rank['asin'][:k].tolist()
    ratings_trainings = training_data

    return recommendations

ratings_by_user = recommender(ratings_trainings,users)


## 結果評估

In [272]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user,ratings_by_user)

0.08813559322033898